In [1]:
import utils.dir_utils as dir_utils
import utils.dict_utils as dict_utils
import utils.ptr_utils as ptr_utils
import utils.constants as constants 
import utils.profit as profit 
import helpers.official as official
import utils.csv_utils as csv_utils

_, input_df = dir_utils.get_data(combined=True)

In [2]:
def setup():
    # (t[constants.TDATE], official.get_name(t), t[constants.TICKER])
    purchases = set()
    
    for _,t in input_df.iterrows():
        if ptr_utils.isvalid(t[constants.TICKER]):
            core =  (t[constants.TDATE], official.get_name(t), t[constants.TICKER])
            
            if t[constants.TYPE] == 'Purchase' and ptr_utils.get_year(t[constants.TDATE]) < 2020: 
                purchases.add(core)

    return purchases

p = setup()


def match(p):
    # week = [name, ticker, tdate_purch, tdate_sale, profit]
    week = []
    month = []
    twomonths = []
    fourmonths = []
    eightmonths = []
    year = []
    i = 0 
    
    for (tdate_purch, name_purch, ticker_purch)  in p:
        i+=1 
        
        # 1 WEEK
        tdate_sale_week = profit.add_to_date(tdate_purch, 1)
        profit_week = profit.share_diff(ticker_purch, tdate_sale_week, tdate_purch)
        if profit_week:
            week.append([name_purch, ticker_purch, tdate_purch, tdate_sale_week, profit_week])
        
        
        # 1 MONTH
        tdate_sale_1month = profit.add_to_date(tdate_purch, 4)
        profit__1month = profit.share_diff(ticker_purch, tdate_sale_1month, tdate_purch)
        if profit__1month:
            month.append([name_purch, ticker_purch, tdate_purch, tdate_sale_1month, profit__1month])
        
        
        # 2 MONTHS
        tdate_sale_2months = profit.add_to_date(tdate_purch, 8)
        profit_2months = profit.share_diff(ticker_purch, tdate_sale_2months, tdate_purch)
        if profit_2months: 
            twomonths.append([name_purch, ticker_purch, tdate_purch, tdate_sale_2months, profit_2months])

        
        # 4 MONTHS
        tdate_sale_4months = profit.add_to_date(tdate_purch, 16)
        profit_4months = profit.share_diff(ticker_purch, tdate_sale_4months, tdate_purch)
        if profit_4months: 
            fourmonths.append([name_purch, ticker_purch, tdate_purch, tdate_sale_4months, profit_4months])

        
        # 8 MONTHS
        tdate_sale_8months = profit.add_to_date(tdate_purch, 32)
        profit_8months = profit.share_diff(ticker_purch, tdate_sale_8months, tdate_purch)
        if profit_8months:
            eightmonths.append([name_purch, ticker_purch, tdate_purch, tdate_sale_8months, profit_8months])

        
        # 12 MONTHS
        tdate_sale_year = profit.add_to_date(tdate_purch, 52)
        profit_year = profit.share_diff(ticker_purch, tdate_sale_year, tdate_purch)
        if profit_year:
            year.append([name_purch, ticker_purch, tdate_purch, tdate_sale_year, profit_year])

        print(i)

    return week, month, twomonths, fourmonths, eightmonths, year

week, month, twomonths, fourmonths, eightmonths, year = match(p)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [ ]:
# week = [name, ticker, tdate_purch, tdate_sale, profit]
dir = dir_utils.makesubdir(constants.path_csv, constants.PROFIT)

x = [(week, "week"),
    (month, "month"),
    (twomonths, "twomonths"),
    (fourmonths, "fourmonths"),
    (eightmonths, "eightmonths"),
    (year, "year")]

print(len(week))
print(len(month))
print(len(twomonths))
print(len(fourmonths))
print(len(eightmonths))
print(len(year))

# official_name, ticker, tdate_purch : [ (tdate_sale : diff) , (tdate_sale : diff), (tdate_sale : diff), ...]
sell_profit = {}
for l, n in x:
    for (official_name, ticker, tdate_purch, tdate_sale, diff) in l:
        sell_profit = dict_utils.increment_list_in_dictionary(sell_profit, (official_name, ticker, tdate_purch), (tdate_sale, diff))

l = []
for (name, ticker, tdate_purch), v in sell_profit.items():
    r = [name, ticker, tdate_purch]
    for (x,y) in v:
        r.append(x)
        r.append(y)
    l.append(r)
    
csv_utils.make_csv_base(dir, "combinedadsatt", [constants.OFFICIAL, constants.TICKER, constants.BUYDATE, "week", constants.PROFIT, "month", constants.PROFIT, "twomonths", "profit", "fourmonths", "profit", "eightmonths", "profit", "year", "profit"], l)